In [1]:
%%writefile requirements.txt

pydantic

Overwriting requirements.txt


In [2]:
!pip install -r requirements.txt

# Pydantic Project

We are going to build a model for an automobile. 

Throughout the course, at the end of each section you will add on to this model, refactor some parts of it, or create related models that will then be used in conjunction with this automobile model when we get to model composition.

To start, you should create an `Automobile` model that contains the following fields:
- `manufacturer`, string, required, not nullable
- `series_name`, string, required, not nullable
- `type_`, string, required, not nullable
- `is_electric`, boolean, defaults to `False`, not nullable
- `manufactured_date`, date, required (hint use `date` from `datetime` module as your field type hint), not nullable
- `base_msrp_usd`, float, required, not nullable
- `vin`, string, required, not nullable
- `number_of_doors`, integer, defaults to `4`, not nullable
- `registration_country`, string, defaults to `None`
- `license_plate`, string, defaults to `None`

Once you have created your model, you should test deserializing and serializing your model and make sure everything works.

In [3]:
from pydantic import BaseModel
from datetime import date

class Automobile(BaseModel):
    manufacturer: str
    series_name: str
    type_: str
    is_electric : bool = False
    manufactured_date: date
    base_msrp_usd : float
    vin : str
    number_of_doors : int = 4
    registration_country : str | None = None
    license_plate : str | None = None
    

You can test your model by deserializing the following input data, and comparing the serialization of each of those models to the provided Python dictionaries.

In other words, test them by doing something like this for both cases:
- create model by deserializing the data
- check the model's serialization to `dict` is equal to the provided expected dictionary

In [4]:
# Python dictionary

data = {
    "manufacturer": "BMW",
    "series_name": "M4",
    "type_": "Convertible",
    "is_electric": False,
    "manufactured_date": "2023-01-01",
    "base_msrp_usd": 93_300,
    "vin": "1234567890",
    "number_of_doors": 2,
    "registration_country": "France",
    "license_plate": "AAA-BBB",
}

data_expected_serialization = {
    'manufacturer': 'BMW',
    'series_name': 'M4',
    'type_': 'Convertible',
    'is_electric': False,
    'manufactured_date': date(2023,1,1),
    'base_msrp_usd': 93_300,
    'vin': '1234567890',
    'number_of_doors': 2,
    'registration_country': 'France',
    'license_plate': 'AAA-BBB',
}

In [5]:
# JSON
data_json = '''
{
    "manufacturer": "BMW",
    "series_name": "M4",
    "type_": "Convertible",
    "manufactured_date": "2023-01-01",
    "base_msrp_usd": 93300,
    "vin": "1234567890"
}
'''

data_json_expected_serialization = {
    'manufacturer': 'BMW',
    'series_name': 'M4',
    'type_': 'Convertible',
    'is_electric': False,
    'manufactured_date': date(2023, 1, 1),
    'base_msrp_usd': 93_300,
    'vin': '1234567890',
    'number_of_doors': 4,
    'registration_country': None,
    'license_plate': None,
}

In [6]:
auto1 = Automobile.model_validate(data)

In [7]:
assert auto1.model_dump() ==  data_expected_serialization

In [8]:
auto2 = Automobile.model_validate_json(data_json)

In [9]:
assert auto2.model_dump() ==  data_json_expected_serialization

Modify your `Automobile` model to implement the following:
- set model to forbid extra fields
- set model to strip whitespace from all string fields
- set model to validate defaults and assignments
- use the enum provided below for the `type_` field

In [10]:
from enum import Enum

class AutomobileType(Enum):
    sedan = "Sedan"
    coupe = "Coupe"
    convertible = "Convertible"
    suv = "SUV"
    truck = "Truck"

You can test your model by serializing and deserializing this data:

In [11]:
data_json = '''
{
    "manufacturer": " BMW ",
    "series_name": " M4 ",
    "type_": "Convertible",
    "manufactured_date": "2023-01-01",
    "base_msrp_usd": 93300,
    "vin": " 1234567890 "
}
'''

data_json_expected_serialization = {
    'manufacturer': 'BMW',
    'series_name': 'M4',
    'type_': AutomobileType.convertible,
    'is_electric': False,
    'manufactured_date': date(2023, 1, 1),
    'base_msrp_usd': 93300.0,
    'vin': '1234567890',
    'number_of_doors': 4,
    'registration_country': None,
    'license_plate': None
}

In [12]:
from pydantic import ConfigDict

class Automobile(BaseModel):
    model_config = ConfigDict(extra='forbid',
                              str_strip_whitespace=True,
                              validate_default=True,
                              validate_assignment=True)
    manufacturer: str
    series_name: str
    type_: AutomobileType
    is_electric : bool = False
    manufactured_date: date
    base_msrp_usd : float
    vin : str
    number_of_doors : int = 4
    registration_country : str | None = None
    license_plate : str | None = None

In [13]:
m = Automobile.model_validate_json(data_json)
assert m.model_dump() == data_json_expected_serialization 

In [14]:
from pydantic import ValidationError
data_invalid = data.copy()
data_invalid['extra_field'] = 'extra'
try:
    Automobile.model_validate(data_invalid)
except ValidationError as e:
    print(e)
    assert "Extra inputs are not permitted" in str(e)

1 validation error for Automobile
extra_field
  Extra inputs are not permitted [type=extra_forbidden, input_value='extra', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/extra_forbidden
